In [1]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from dlomix.losses import masked_spectral_distance, masked_pearson_correlation_distance
from dlomix.data import FragmentIonIntensityDataset

import os

2024-10-22 11:18:38.514181: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-22 11:18:38.568234: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-22 11:18:38.568285: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-22 11:18:38.570076: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-22 11:18:38.578700: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-22 11:18:38.579829: I tensorflow/core/platform/cpu_feature_guard.cc:1


Avaliable feature extractors are (use the key of the following dict and pass it to features_to_extract in the Dataset Class):
{
   "atom_count": "Atom count of PTM.",
   "delta_mass": "Delta mass of PTM.",
   "mod_gain": "Gain of atoms due to PTM.",
   "mod_loss": "Loss of atoms due to PTM.",
   "red_smiles": "Reduced SMILES representation of PTM."
}.
When writing your own feature extractor, you can either
    (1) use the FeatureExtractor class or
    (2) write a function that can be mapped to the Hugging Face dataset.
In both cases, you can access the parsed sequence information from the dataset using the following keys, which all provide python lists:
    - _parsed_sequence: parsed sequence
    - _n_term_mods: N-terminal modifications
    - _c_term_mods: C-terminal modifications



In [55]:
model_name = 'b_r_single_both_d5_static_0.0005_TeG'

model = keras.saving.load_model('/cmnfs/proj/prosit_astral/saved_models/final_models/%s.keras' % model_name,
                                custom_objects={'masked_spectral_distance': masked_spectral_distance, 'masked_pearson_correlation_distance': masked_pearson_correlation_distance}, 
                                compile=True, safe_mode=True)
model.summary()

Model: "transformer_model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_25 (Dense)            multiple                  6656      
                                                                 
 dense_26 (Dense)            multiple                  3328      
                                                                 
 dense_27 (Dense)            multiple                  131584    
                                                                 
 trans_block_25 (TransBlock  multiple                  524545    
 )                                                               
                                                                 
 trans_block_26 (TransBlock  multiple                  524545    
 )                                                               
                                                                 
 trans_block_27 (TransBlock  multiple          

In [56]:
int_data = FragmentIonIntensityDataset.load_from_disk("/cmnfs/proj/prosit_astral/datasets/_dlomix_saved/full_dataset")

In [57]:
eval_batches = [m for m in int_data.tensor_train_data.take(3)]

pred_ast = []
pred_tof = []
pred_lum = []

true_ast = []
true_tof = []
true_lum = []


for j in range(len(eval_batches)):

    #eval_batches[j][0]["method_nr_oh"] = tf.constant([[1., 0.]] * 1024, dtype=tf.float32)

    pred = model.predict(eval_batches[j][0])

    batch_size = pred.shape[0]

    mach = np.split(eval_batches[j][0]['machine_oh'], batch_size, axis=0)
    pred = np.split(pred, batch_size, axis=0)
    true = np.split(eval_batches[j][1], batch_size, axis=0)

    for i in range(batch_size):

        if np.argmax(mach[i]) == 0:
            pred_ast.append(pred[i])
            true_ast.append(true[i])
        elif np.argmax(mach[i]) == 1:
            pred_tof.append(pred[i])
            true_tof.append(true[i])
        elif np.argmax(mach[i]) == 2:
            pred_lum.append(pred[i])
            true_lum.append(true[i])

pred_ast = np.concatenate(pred_ast, axis=0)
pred_tof = np.concatenate(pred_tof, axis=0)
pred_lum = np.concatenate(pred_lum, axis=0)

true_ast = np.concatenate(true_ast, axis=0)
true_tof = np.concatenate(true_tof, axis=0)
true_lum = np.concatenate(true_lum, axis=0)

print('AST: ', tf.reduce_mean( masked_spectral_distance(true_ast, pred_ast) ))
print('TOF: ', tf.reduce_mean( masked_spectral_distance(true_tof, pred_tof) ))
print('LUM: ', tf.reduce_mean( masked_spectral_distance(true_lum, pred_lum) ))

RuntimeError: pybind11::error_already_set: MISMATCH of original and normalized active exception types: ORIGINAL _NotOkStatusException REPLACED BY KeyboardInterrupt: <EMPTY MESSAGE>

At:
  /nfs/home/students/d.lochert/miniconda3/envs/astral/lib/python3.10/site-packages/tensorflow/python/eager/core.py(42): __init__
  /nfs/home/students/d.lochert/miniconda3/envs/astral/lib/python3.10/site-packages/tensorflow/python/ops/gen_dataset_ops.py(3024): iterator_get_next
  /nfs/home/students/d.lochert/miniconda3/envs/astral/lib/python3.10/site-packages/tensorflow/python/data/ops/iterator_ops.py(773): _next_internal
  /nfs/home/students/d.lochert/miniconda3/envs/astral/lib/python3.10/site-packages/tensorflow/python/data/ops/iterator_ops.py(810): __next__
  /tmp/ipykernel_1999882/1637051582.py(1): <listcomp>
  /tmp/ipykernel_1999882/1637051582.py(1): <module>
  /nfs/home/students/d.lochert/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py(3508): run_code
  /nfs/home/students/d.loc

In [44]:
eval_batches[0][0]["method_nr_oh"]

tf.constant([[0., 1.]] * 1024, dtype=tf.float32)

<tf.Tensor: shape=(1024, 2), dtype=float32, numpy=
array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)>

In [37]:
print('AST: ', tf.reduce_mean( masked_spectral_distance(true_ast, pred_ast) ))
print('TOF: ', tf.reduce_mean( masked_spectral_distance(true_tof, pred_tof) ))
print('LUM: ', tf.reduce_mean( masked_spectral_distance(true_lum, pred_lum) ))

AST:  tf.Tensor(0.5086041, shape=(), dtype=float32)
TOF:  tf.Tensor(0.7088966, shape=(), dtype=float32)
LUM:  tf.Tensor(0.68091726, shape=(), dtype=float32)


In [52]:
import pandas as pd
import numpy as np

tmp = pd.DataFrame({
    'model': np.repeat("single", len(pred_ast)),
    'dataset': ["AST"] * len(pred_ast),
    'loss': 1 - tf.concat([masked_spectral_distance(true_ast, pred_ast)], axis=0),
})

all_data = pd.concat([all_data, tmp], axis=0)

all_data.groupby(['dataset', 'model'])['loss'].median()

dataset  model 
AST      full      0.829409
         single    0.823133
LUM      full      0.926071
         single    0.927606
TOF      full      0.880708
         single    0.857731
Name: loss, dtype: float32

In [30]:
tmp
tmp.groupby(['dataset', 'model'])['loss'].median()

dataset  model 
LUM      single    0.927606
Name: loss, dtype: float32

In [53]:
# save all_data
all_data.to_csv('~/projects/astral/dlomix-astral/plots/full_vs_balanced/data.csv', index=False)